In [3]:
import numpy as np
import pandas as pd
import os
# Lista de posibles rutas
addresses = [
    'C:/Users/RONALD Q/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/RDIS.xlsx',
    'C:/Users/roquispec/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/RDIS.xlsx',
    'C:/Users/mticllacu/OneDrive - LUZ DEL SUR S.A.A/Archivos de Ronald Quispe Ocaña - ProyectoRyD_V2/Basededatos/RDIS.xlsx'
]

df = None
for path in addresses:
    if os.path.exists(path):   # verifica si existe
        df = pd.read_excel(path,header=1)
        print(f"✅ Archivo cargado desde: {path}")
        break

if df is None:
    raise FileNotFoundError("❌ No se encontró el archivo en ninguna de las rutas especificadas.")


df = df.drop(columns=df.columns[0])
# Aseguramos que FECHA sea datetime
df["FECHA"] = pd.to_datetime(df["FECHA"], errors="coerce")
df

✅ Archivo cargado desde: C:/Users/roquispec/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/RDIS.xlsx


,SERIE,FECHA,H-X 1 [%] ONAF 2,H-X 14 [%] ONAF 2,H-X 27 [%] ONAF 2,H-Y 1 [%],H-Y 14 [%],H-Y 27 [%],X-Y 1 [%],X-Y 2 [%],X-Y 3 [%]
0,146660T3,2022-05-12,11.720,11.070,10.670,NaN,NaN,NaN,NaN,NaN,NaN
1,146660T3,2018-02-20,11.438,5.847,9.928,NaN,NaN,NaN,NaN,NaN,NaN
2,146660T3,2016-07-31,6.925,11.276,7.360,NaN,NaN,NaN,NaN,NaN,NaN
3,146660T3,2015-06-06,5.967,11.012,6.140,NaN,NaN,NaN,NaN,NaN,NaN
4,364076,2022-06-18,8.346,9.233,8.207,NaN,NaN,NaN,NaN,NaN,NaN
5,364076,2017-11-30,10.678,8.843,7.480,NaN,NaN,NaN,NaN,NaN,NaN
6,364076,2015-08-03,11.247,7.075,10.350,NaN,NaN,NaN,NaN,NaN,NaN
7,364076,2013-12-30,11.627,7.480,11.131,NaN,NaN,NaN,NaN,NaN,NaN
8,230531-01,2024-07-16,9.404,5.775,8.360,8.407,9.307,9.156,7.934,7.023,7.495
9,230531-01,2018-04-14,9.783,7.559,7.661,10.865,8.561,10.881,8.403,10.383,9.971


In [4]:
cap_cols = df.iloc[:,2:]

for col in cap_cols:
    # valor inicial por SERIE en la fecha mínima
    ref = df.groupby("SERIE").apply(lambda g: g.loc[g["FECHA"].idxmin(), col])
    ref_mapped = df["SERIE"].map(ref)
     # variación porcentual
    delta = abs((df[col] - ref_mapped) / ref_mapped) * 100
    delta = delta.replace([np.inf, -np.inf], np.nan)

    # marcar casos inválidos
    mask = df[col].isna() | ref_mapped.isna() | (ref_mapped == 0) | (df[col] == 0)
    delta[mask] = np.nan

    # agregar columna nueva
    df[f"{col}_Delta"] = delta
df

C:\Users\roquispec\AppData\Local\Temp\ipykernel_36236\1282297196.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ref = df.groupby("SERIE").apply(lambda g: g.loc[g["FECHA"].idxmin(), col])
C:\Users\roquispec\AppData\Local\Temp\ipykernel_36236\1282297196.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ref = df.groupby("SERIE").apply(lambda g: g.loc[g["FECHA"].idxmin(), col])
C:\Users\roquispec\AppData\Local\Tem

,SERIE,FECHA,H-X 1 [%] ONAF 2,H-X 14 [%] ONAF 2,H-X 27 [%] ONAF 2,H-Y 1 [%],H-Y 14 [%],H-Y 27 [%],X-Y 1 [%],X-Y 2 [%],X-Y 3 [%],H-X 1 [%] ONAF 2_Delta,H-X 14 [%] ONAF 2_Delta,H-X 27 [%] ONAF 2_Delta,H-Y 1 [%]_Delta,H-Y 14 [%]_Delta,H-Y 27 [%]_Delta,X-Y 1 [%]_Delta,X-Y 2 [%]_Delta,X-Y 3 [%]_Delta
0,146660T3,2022-05-12,11.720,11.070,10.670,NaN,NaN,NaN,NaN,NaN,NaN,96.413608,0.526698,73.778502,NaN,NaN,NaN,NaN,NaN,NaN
1,146660T3,2018-02-20,11.438,5.847,9.928,NaN,NaN,NaN,NaN,NaN,NaN,91.687615,46.903378,61.693811,NaN,NaN,NaN,NaN,NaN,NaN
2,146660T3,2016-07-31,6.925,11.276,7.360,NaN,NaN,NaN,NaN,NaN,NaN,16.054969,2.397385,19.869707,NaN,NaN,NaN,NaN,NaN,NaN
3,146660T3,2015-06-06,5.967,11.012,6.140,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
4,364076,2022-06-18,8.346,9.233,8.207,NaN,NaN,NaN,NaN,NaN,NaN,28.218801,23.435829,26.268979,NaN,NaN,NaN,NaN,NaN,NaN
5,364076,2017-11-30,10.678,8.843,7.480,NaN,NaN,NaN,NaN,NaN,NaN,8.162037,18.221925,32.800287,NaN,NaN,NaN,NaN,NaN,NaN
6,364076,2015-08-03,11.247,7.075,10.350,NaN,NaN,NaN,NaN,NaN,NaN,3.268255,5.414439,7.016441,NaN,NaN,NaN,NaN,NaN,NaN
7,364076,2013-12-30,11.627,7.480,11.131,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
8,230531-01,2024-07-16,9.404,5.775,8.360,8.407,9.307,9.156,7.934,7.023,7.495,14.679731,46.204006,33.717210,40.632319,80.228505,64.676259,30.837731,23.555023,13.010678
9,230531-01,2018-04-14,9.783,7.559,7.661,10.865,8.561,10.881,8.403,10.383,9.971,11.241154,29.585468,22.536788,81.749749,65.782339,95.701439,38.571900,13.018396,15.726555


In [5]:
# Calcular el máximo de las columnas que terminan en 'Delta' para cada fila
cols_delta = [col for col in df.columns if col.endswith('Delta')]
df['Max_Dis'] = df[cols_delta].max(axis=1)
df

,SERIE,FECHA,H-X 1 [%] ONAF 2,H-X 14 [%] ONAF 2,H-X 27 [%] ONAF 2,H-Y 1 [%],H-Y 14 [%],H-Y 27 [%],X-Y 1 [%],X-Y 2 [%],...,H-X 1 [%] ONAF 2_Delta,H-X 14 [%] ONAF 2_Delta,H-X 27 [%] ONAF 2_Delta,H-Y 1 [%]_Delta,H-Y 14 [%]_Delta,H-Y 27 [%]_Delta,X-Y 1 [%]_Delta,X-Y 2 [%]_Delta,X-Y 3 [%]_Delta,Max_Dis
0,146660T3,2022-05-12,11.720,11.070,10.670,NaN,NaN,NaN,NaN,NaN,...,96.413608,0.526698,73.778502,NaN,NaN,NaN,NaN,NaN,NaN,96.413608
1,146660T3,2018-02-20,11.438,5.847,9.928,NaN,NaN,NaN,NaN,NaN,...,91.687615,46.903378,61.693811,NaN,NaN,NaN,NaN,NaN,NaN,91.687615
2,146660T3,2016-07-31,6.925,11.276,7.360,NaN,NaN,NaN,NaN,NaN,...,16.054969,2.397385,19.869707,NaN,NaN,NaN,NaN,NaN,NaN,19.869707
3,146660T3,2015-06-06,5.967,11.012,6.140,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
4,364076,2022-06-18,8.346,9.233,8.207,NaN,NaN,NaN,NaN,NaN,...,28.218801,23.435829,26.268979,NaN,NaN,NaN,NaN,NaN,NaN,28.218801
5,364076,2017-11-30,10.678,8.843,7.480,NaN,NaN,NaN,NaN,NaN,...,8.162037,18.221925,32.800287,NaN,NaN,NaN,NaN,NaN,NaN,32.800287
6,364076,2015-08-03,11.247,7.075,10.350,NaN,NaN,NaN,NaN,NaN,...,3.268255,5.414439,7.016441,NaN,NaN,NaN,NaN,NaN,NaN,7.016441
7,364076,2013-12-30,11.627,7.480,11.131,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
8,230531-01,2024-07-16,9.404,5.775,8.360,8.407,9.307,9.156,7.934,7.023,...,14.679731,46.204006,33.717210,40.632319,80.228505,64.676259,30.837731,23.555023,13.010678,80.228505
9,230531-01,2018-04-14,9.783,7.559,7.661,10.865,8.561,10.881,8.403,10.383,...,11.241154,29.585468,22.536788,81.749749,65.782339,95.701439,38.571900,13.018396,15.726555,95.701439


In [6]:
# Asignar puntaje DIS según el valor de Max_Dis y la tabla IEEE C57.152-2013
conditions = [
    df['Max_Dis'] > 3,
    (df['Max_Dis'] > 2) & (df['Max_Dis'] <= 3),
    df['Max_Dis'] <= 2
]
choices = [5, 3, 1]
df['DIS'] = np.select(conditions, choices, default=np.nan)
df

,SERIE,FECHA,H-X 1 [%] ONAF 2,H-X 14 [%] ONAF 2,H-X 27 [%] ONAF 2,H-Y 1 [%],H-Y 14 [%],H-Y 27 [%],X-Y 1 [%],X-Y 2 [%],...,H-X 14 [%] ONAF 2_Delta,H-X 27 [%] ONAF 2_Delta,H-Y 1 [%]_Delta,H-Y 14 [%]_Delta,H-Y 27 [%]_Delta,X-Y 1 [%]_Delta,X-Y 2 [%]_Delta,X-Y 3 [%]_Delta,Max_Dis,DIS
0,146660T3,2022-05-12,11.720,11.070,10.670,NaN,NaN,NaN,NaN,NaN,...,0.526698,73.778502,NaN,NaN,NaN,NaN,NaN,NaN,96.413608,5.0
1,146660T3,2018-02-20,11.438,5.847,9.928,NaN,NaN,NaN,NaN,NaN,...,46.903378,61.693811,NaN,NaN,NaN,NaN,NaN,NaN,91.687615,5.0
2,146660T3,2016-07-31,6.925,11.276,7.360,NaN,NaN,NaN,NaN,NaN,...,2.397385,19.869707,NaN,NaN,NaN,NaN,NaN,NaN,19.869707,5.0
3,146660T3,2015-06-06,5.967,11.012,6.140,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.0
4,364076,2022-06-18,8.346,9.233,8.207,NaN,NaN,NaN,NaN,NaN,...,23.435829,26.268979,NaN,NaN,NaN,NaN,NaN,NaN,28.218801,5.0
5,364076,2017-11-30,10.678,8.843,7.480,NaN,NaN,NaN,NaN,NaN,...,18.221925,32.800287,NaN,NaN,NaN,NaN,NaN,NaN,32.800287,5.0
6,364076,2015-08-03,11.247,7.075,10.350,NaN,NaN,NaN,NaN,NaN,...,5.414439,7.016441,NaN,NaN,NaN,NaN,NaN,NaN,7.016441,5.0
7,364076,2013-12-30,11.627,7.480,11.131,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.0
8,230531-01,2024-07-16,9.404,5.775,8.360,8.407,9.307,9.156,7.934,7.023,...,46.204006,33.717210,40.632319,80.228505,64.676259,30.837731,23.555023,13.010678,80.228505,5.0
9,230531-01,2018-04-14,9.783,7.559,7.661,10.865,8.561,10.881,8.403,10.383,...,29.585468,22.536788,81.749749,65.782339,95.701439,38.571900,13.018396,15.726555,95.701439,5.0


In [ ]:
# Crear tabla final solo con SERIE, FECHA, DIS y las columnas que no terminan en 'Delta' ni son 'Max_Dis'
cols_final = [col for col in df.columns if not col.endswith('Delta') and col != 'Max_Dis']
# Aseguramos el orden: SERIE, FECHA, DIS primero
cols_final = ['SERIE', 'FECHA', 'DIS'] + [col for col in cols_final if col not in ['SERIE', 'FECHA', 'DIS']]
df_full = df[cols_final]
df_full

,SERIE,FECHA,DIS,H-X 1 [%] ONAF 2,H-X 14 [%] ONAF 2,H-X 27 [%] ONAF 2,H-Y 1 [%],H-Y 14 [%],H-Y 27 [%],X-Y 1 [%],X-Y 2 [%],X-Y 3 [%]
0,146660T3,2022-05-12,5.0,11.720,11.070,10.670,NaN,NaN,NaN,NaN,NaN,NaN
1,146660T3,2018-02-20,5.0,11.438,5.847,9.928,NaN,NaN,NaN,NaN,NaN,NaN
2,146660T3,2016-07-31,5.0,6.925,11.276,7.360,NaN,NaN,NaN,NaN,NaN,NaN
3,146660T3,2015-06-06,1.0,5.967,11.012,6.140,NaN,NaN,NaN,NaN,NaN,NaN
4,364076,2022-06-18,5.0,8.346,9.233,8.207,NaN,NaN,NaN,NaN,NaN,NaN
5,364076,2017-11-30,5.0,10.678,8.843,7.480,NaN,NaN,NaN,NaN,NaN,NaN
6,364076,2015-08-03,5.0,11.247,7.075,10.350,NaN,NaN,NaN,NaN,NaN,NaN
7,364076,2013-12-30,1.0,11.627,7.480,11.131,NaN,NaN,NaN,NaN,NaN,NaN
8,230531-01,2024-07-16,5.0,9.404,5.775,8.360,8.407,9.307,9.156,7.934,7.023,7.495
9,230531-01,2018-04-14,5.0,9.783,7.559,7.661,10.865,8.561,10.881,8.403,10.383,9.971
